# EDA

- RMSE : 23724.4207

In [ ]:
!pip install eli5==0.13.0

# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
# 열 다 보이게 설정
pd.set_option('display.max_columns', None)
# 행 다 보이게 설정
pd.set_option('display.max_rows', None)

In [ ]:
# 전처리 위해 데이터 합치고 확인

train['data'] = 0
test['data'] = 1
concat = pd.concat([train, test])

print(concat.shape)
print(concat['data'].value_counts())

In [ ]:
# 결측치 탐색 및 보간

# 열 전체를 넣고 스캔하기

for col in concat.columns:
    nunique = concat[col].nunique(dropna=False)
    missing_ratio = concat[col].isna().mean()
    missing_count = concat[col].isnull().sum()
    col_type = concat.dtypes[col]
    print(f"📌 {col:30} | 데이터타입: {col_type} | 고유값: {nunique:6} | 결측개수: {missing_count} | 결측률: {missing_ratio:.2%}")

In [ ]:
# 결측치는 아닌데 의미 없는 형식적 값 찾기

def detect_fake_nulls(df, suspect_values=['-', ' ', '', '.', '없음', 'nan']):
    result = {}
    for col in df.columns:
        if concat[col].dtype == 'object':
            val_counts = concat[col].value_counts(dropna=False)
            found = val_counts[val_counts.index.isin(suspect_values)]
            if not found.empty:
                result[col] = found
    return result

fake_nulls = detect_fake_nulls(concat)
for col, vals in fake_nulls.items():
    print(f"🔎 {col} 컬럼에서 의미 없는 값 발견:")
    print(vals)
    print()

In [ ]:
# 의미없는 값 결측치로 인식시키고 결측률 보기
# 열 전체를 넣고 스캔하기

for col in concat.columns:
    nunique = concat[col].nunique(dropna=False)
    missing_ratio = concat[col].isna().mean()
    missing_count = concat[col].isnull().sum()
    col_type = concat.dtypes[col]
    print(f"📌 {col:30} | 데이터타입: {col_type} | 고유값: {nunique:6} | 결측개수: {missing_count} | 결측률: {missing_ratio:.2%}")

In [ ]:
# 결측 채우고 info 본 후에 데이터 타입 변환
# 본번, 부번의 경우 float로 되어있지만 범주형 변수의 의미를 가지므로 object(string) 형태로 바꾸어주고 아래 작업을 진행하겠습니다.
concat_select['본번'] = concat_select['본번'].astype('str')
concat_select['부번'] = concat_select['부번'].astype('str')

In [ ]:
def null_summary(df, columns):
    result = pd.DataFrame({
        '결측 개수': df[columns].isnull().sum(),
        '결측 비율(%)': df[columns].isnull().mean() * 100
    })
    return result[result['결측 개수'] > 0].sort_values('결측 비율(%)', ascending=False)

print("📊 연속형 변수 결측치 요약")
display(null_summary(concat_select, con_columns))

print("📊 범주형 변수 결측치 요약")
display(null_summary(concat_select, cat_columns))


In [ ]:
# 결측치 탐지 및 처리 전 연속형 변수 상관관계 보기

# 1. 수치형 변수만 선택
numeric_cols = concat_select.select_dtypes(include=['float64', 'int64']).columns

# 2. 상관관계 계산
corr = concat_select[numeric_cols].corr()

# 3. 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("📊 수치형 변수 간 상관관계")
plt.show()

In [ ]:
# 상관관계 기반으로 중복 feature 쌍 탐지 및 삭제 후보 추천

# 연속형 변수만 추출
numeric_cols = con_columns  # ← 너가 나눠둔 연속형 변수 리스트

# 상관관계 행렬 (절댓값 기준)
corr_matrix = concat_select[numeric_cols].corr().abs()

# 상삼각 행렬로 중복 제거
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 상관계수 0.9 초과인 변수쌍 추출
high_corr_pairs = [(col, row, upper.loc[row, col])
                   for col in upper.columns
                   for row in upper.index
                   if pd.notnull(upper.loc[row, col]) and upper.loc[row, col] > 0.7]

# 출력
for col1, col2, score in sorted(high_corr_pairs, key=lambda x: -x[2]):
    print(f"🔁 {col1} ↔ {col2} : 상관계수 = {score:.2f}")


In [ ]:
# 상관관계 보고 pca와 제거 진행 후 리스트 초기화 및 지정
# 범주형 변수 관계 보기 전 확인
# 숫자형 분리 pd.api.types.is_numeric_dtype
con_columns2 = []
cat_columns2 = []

for column in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[column]):
        con_columns2.append(column)
    else:
        cat_columns2.append(column)

print("연속형 변수:", con_columns2)
print("범주형 변수:", cat_columns2)

def null_summary(df, columns):
    result = pd.DataFrame({
        '결측 개수': df[columns].isnull().sum(),
        '결측 비율(%)': df[columns].isnull().mean() * 100
    })
    return result[result['결측 개수'] > 0].sort_values('결측 비율(%)', ascending=False)

print("📊 연속형 변수 결측치 요약")
display(null_summary(concat_select, con_columns2))

print("📊 범주형 변수 결측치 요약")
display(null_summary(concat_select, cat_columns2))


In [ ]:
# 범주형 feature들 관계 보기
# 보기 전 결측값은 임의로 채움

from scipy.stats import chi2_contingency

# Cramér's V 계산 함수
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))

# 범주형 변수 리스트
cat_cols = cat_columns2  # 이미 나눈 리스트

# 결과 저장용 매트릭스
cramer_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols)

for col1 in cat_cols:
    for col2 in cat_cols:
        if col1 == col2:
            cramer_matrix.loc[col1, col2] = 1.0
        else:
            try:
                cramer_matrix.loc[col1, col2] = cramers_v(concat_select[col1], concat_select[col2])
            except:
                cramer_matrix.loc[col1, col2] = np.nan

plt.figure(figsize=(14, 12))
sns.heatmap(cramer_matrix.astype(float), cmap='coolwarm', annot=False)
plt.title("범주형 변수 간 Cramér's V (상관관계 유사도)")
plt.show()


In [ ]:
# 유니크 값이 낮으면 값이 다양하지 않아 유사성이 높을 수 있다고 함
# 변주형 중 unique값이 낮은 것

for col in cat_columns2:
    print(f"{col}: {concat_select[col].nunique()}")


In [ ]:
# 크래머스 브이 수치로 보기

threshold = 0.8
high_corr_pairs = []

for i in range(len(cat_cols)):
    for j in range(i + 1, len(cat_cols)):
        col1, col2 = cat_cols[i], cat_cols[j]
        val = cramer_matrix.loc[col1, col2]
        if pd.notnull(val) and float(val) >= threshold:
            high_corr_pairs.append((col1, col2, float(val)))

for col1, col2, score in sorted(high_corr_pairs, key=lambda x: -x[2]):
    print(f"🔁 {col1} ↔ {col2} : Cramér's V = {score:.2f}")


In [ ]:
# 최종 확인
# 빈 리스트 초기화
con_columns_final = []
cat_columns_final = []

# 현재 concat_filtered 기준으로 데이터타입 분리
for col in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[col]):
        con_columns_final.append(col)
    else:
        cat_columns_final.append(col)

# 결과 출력
print("✅ 연속형 변수:", con_columns_final)
print("✅ 범주형 변수:", cat_columns_final)
print(f"📊 총 연속형 변수 개수: {len(con_columns_final)}")
print(f"📁 총 범주형 변수 개수: {len(cat_columns_final)}")


### 파생변수 생성 후 살펴보기

In [ ]:
concat_select[['지하철_1000m내_개수', '지하철_1000m내_이름목록', '지하철_최단거리_km','지하철_최단거리_역명']]

In [ ]:
# 지하철과 버스 데이터로 파생변수 생성 후 전체 살피기

concat_select.info()

In [ ]:
# 리스트 초기화
con_columns_final = []
cat_columns_final = []

# concat_select 기준으로 분리
for col in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[col]):
        con_columns_final.append(col)
    else:
        cat_columns_final.append(col)

# 결과 출력
print("✅ 연속형 변수:", con_columns_final)
print("✅ 범주형 변수:", cat_columns_final)
print(f"📊 총 연속형 변수 개수: {len(con_columns_final)}")
print(f"📁 총 범주형 변수 개수: {len(cat_columns_final)}")


In [ ]:
# 추가한 변수들 상관관계 보기

correlation_with_target = concat_select[con_columns_final].corr()['target'].sort_values(ascending=False)
print(correlation_with_target)

plt.figure(figsize=(10, 8))
sns.heatmap(concat_select[con_columns_final].corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("연속형 변수 상관관계 히트맵")
plt.show()


In [ ]:
# 추가한 변수(범주형) 관계 보기

# 범주형 feature들 관계 보기

from scipy.stats import chi2_contingency

# Cramér's V 계산 함수
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))

# 범주형 변수 리스트
cat_cols = cat_columns2 = cat_columns_final  # 이미 나눈 리스트

# 결과 저장용 매트릭스
cramer_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols)

for col1 in cat_cols:
    for col2 in cat_cols:
        if col1 == col2:
            cramer_matrix.loc[col1, col2] = 1.0
        else:
            try:
                cramer_matrix.loc[col1, col2] = cramers_v(concat_select[col1], concat_select[col2])
            except:
                cramer_matrix.loc[col1, col2] = np.nan

plt.figure(figsize=(14, 12))
sns.heatmap(cramer_matrix.astype(float), cmap='coolwarm', annot=False)
plt.title("범주형 변수 간 Cramér's V (상관관계 유사도)")
plt.show()


In [ ]:
threshold = 0.8
high_corr_pairs = []

for i in range(len(cat_cols)):
    for j in range(i + 1, len(cat_cols)):
        col1, col2 = cat_cols[i], cat_cols[j]
        val = cramer_matrix.loc[col1, col2]
        if pd.notnull(val) and float(val) >= threshold:
            high_corr_pairs.append((col1, col2, float(val)))

for col1, col2, score in sorted(high_corr_pairs, key=lambda x: -x[2]):
    print(f"🔁 {col1} ↔ {col2} : Cramér's V = {score:.2f}")
